In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2025-02-15 19:00:57.576641: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-15 19:00:57.588143: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739626257.604652   34646 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739626257.609507   34646 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-15 19:00:57.621803: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
import os

In [3]:
data_path=os.path.join("..","Data","lstm_model_data_v1.csv")

In [4]:
data=pd.read_csv(data_path)

In [5]:
data.dropna(inplace=True)

In [6]:
data

,processed_text_upto_300_words,label
0,students faculty georgetown law school gathere...,1
1,beirut parliament group lebanon hezbollah said...,1
2,kabul us ambassador afghanistan said monday wo...,1
3,berlin chancellor angela merkel friday said ge...,1
4,easy blame white man way accountable actions l...,0
...,...,...
44892,donald trump literally used national prayer br...,0
44893,time drag donald trump office act soon constit...,0
44895,donald trump misogynistic behavior towards wom...,0
44896,president donald trump declined friday explain...,1


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44196 entries, 0 to 44897
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   processed_text_upto_300_words  44196 non-null  object
 1   label                          44196 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [8]:
data_X=data["processed_text_upto_300_words"]
data_Y=data["label"]

## Tokenize Text

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [10]:
tokenizer=Tokenizer()

In [11]:
tokenizer.fit_on_texts(data_X)

In [12]:
tokenized_text=tokenizer.texts_to_sequences(data_X)

In [13]:
vocab_length=len(tokenizer.word_index)

In [14]:
vocab_length

108786

In [15]:
word_index=tokenizer.word_index

## Pad Sequences

In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
padded_text=pad_sequences(tokenized_text,maxlen=300,truncating="post",padding="post")

In [18]:
padded_text.shape

(44196, 300)

## Embedding matrix

In [19]:
embedding_matrix=np.zeros((vocab_length+1,100))

In [20]:
embedding_path=os.path.join("..","Data","glove.6B.100d.txt")

In [21]:
embed_dict={}
with open(embedding_path,'r',encoding='utf-8') as file:
    for line in file:
        vector=line.split()
        word=vector[0]
        embed=np.asarray(vector[1:])
        embed_dict[word]=embed

for word,index in word_index.items():
    if word in embed_dict.keys():
        embedding_matrix[index]=embed_dict[word]
        

In [22]:
embedding_matrix.shape

(108787, 100)

### Important Note: There will be 6 Lstm models and each having 60 words per instance.

## Model Building

In [23]:
from tensorflow.keras.layers import Input


In [24]:
from multi_lstm_layer import MultiLSTMLayer

In [25]:
basic_text_processing_layer_config={
    "lstm_layer_config":[
        {
            "units":8,
            "activation":"relu",
            "return_sequences":True,
            "bidirection":True
        },
        {
            "units":32,
            "activation":"relu",
            "return_sequences":True,
            "bidirection":False
        },
        {
            "units":64,
            "activation":"relu",
            "return_sequences":False,
            "bidirection":False
        }
        
    ],
    "dense_layer_config":[
        {
           "units":128,
           "activation":"relu",
           "kernel_initializer":"he_uniform"
        },
        {
           "units":256,
           "activation":"relu",
           "kernel_initializer":"he_uniform"
        }
        
    ]
}

In [26]:
inp=Input(shape=(300,),dtype=tf.int32)

In [27]:
x=MultiLSTMLayer(
    basic_text_processing_layer_config=basic_text_processing_layer_config,
    embedding_matrix=embedding_matrix,
    num_models=6,
    vocab_size=vocab_length+1,
    embedding_output_dim=100
)(inp)

2025-02-15 19:01:10.266283: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [28]:
from tensorflow.keras.models import Model

In [29]:
model=Model(inputs=inp,outputs=x)

In [30]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ multi_lstm_layer                │ (None, 1)              │    18,449,349 │
│ (MultiLSTMLayer)                │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,449,349 (70.38 MB)

 Trainable params: 18,449,349 (70.38 MB)

 Non-trainable params: 0 (0.00 B)

## Compile Model

In [31]:
from tensorflow.keras.optimizers import Adam

In [32]:
from tensorflow.keras.losses import BinaryCrossentropy

In [33]:
from tensorflow.keras.metrics import Accuracy

In [34]:
optimizer=Adam(
    learning_rate=0.01,
    beta_1=0.96,
    beta_2=0.98
)

In [35]:
# loss_fn=BinaryCrossentropy()

In [36]:
# metric=Accuracy()

In [37]:
model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

## Model Training

In [38]:
X=padded_text
X.shape

(44196, 300)

In [39]:
Y=np.array(data_Y)
Y.shape

(44196,)

In [40]:
batch_size=100
epochs=30

In [41]:
history=model.fit(X,Y,batch_size=batch_size,epochs=epochs)

Epoch 1/30
442/442 ━━━━━━━━━━━━━━━━━━━━ 135s 267ms/step - accuracy: 0.5137 - loss: 1.5494
Epoch 2/30
442/442 ━━━━━━━━━━━━━━━━━━━━ 117s 265ms/step - accuracy: 0.5175 - loss: 0.6925
Epoch 3/30
442/442 ━━━━━━━━━━━━━━━━━━━━ 119s 270ms/step - accuracy: 0.5160 - loss: 0.6928
Epoch 4/30
442/442 ━━━━━━━━━━━━━━━━━━━━ 120s 271ms/step - accuracy: 0.5154 - loss: 0.6928
Epoch 5/30
442/442 ━━━━━━━━━━━━━━━━━━━━ 119s 269ms/step - accuracy: 0.5196 - loss: 0.6924
Epoch 6/30
442/442 ━━━━━━━━━━━━━━━━━━━━ 117s 264ms/step - accuracy: 0.5186 - loss: 0.6927
Epoch 7/30
442/442 ━━━━━━━━━━━━━━━━━━━━ 120s 272ms/step - accuracy: 0.5170 - loss: 0.6927
Epoch 8/30
442/442 ━━━━━━━━━━━━━━━━━━━━ 122s 276ms/step - accuracy: 0.5150 - loss: 0.6929
Epoch 9/30
442/442 ━━━━━━━━━━━━━━━━━━━━ 121s 273ms/step - accuracy: 0.5165 - loss: 0.6927
Epoch 10/30
442/442 ━━━━━━━━━━━━━━━━━━━━ 118s 267ms/step - accuracy: 0.5160 - loss: 0.6927
Epoch 11/30
442/442 ━━━━━━━━━━━━━━━━━━━━ 20120s 46s/step - accuracy: 0.5127 - loss: 0.6930
Epoch 12

In [45]:
import os

In [53]:
lstm_model_path=os.path.join("..","Models","lstm_model.keras");

In [54]:
model.save(lstm_model_path)

In [42]:
# for layer in model.layers:
#     layer.trainable=True

In [43]:
# for epoch in range(epochs):
#     print(f"Epoch:{epoch+1} started..")
    
#     dataset=tf.data.Dataset.from_tensor_slices((X,Y))
#     dataset=dataset.shuffle(buffer_size=1024).batch(batch_size)
    
    
#     for step,(data_X,data_Y) in enumerate(dataset):
#         with tf.GradientTape() as tape:
#             predictions=model(data_X,training=True)
#             loss=loss_fn(data_Y,predictions)
            
#         # print(f"Pred:{predictions}")
#         # print(f"Loss:{loss}")
        
#         gradients=tape.gradient(loss,model.trainable_variables)
#         optimizer.apply_gradients(zip(gradients,model.trainable_variables))
        
#         metric.update_state(data_Y, tf.argmax(predictions, axis=-1))
        
#         if step%20==0:
#             print(f"Epoch:{epoch+1} Step:{step+1} Accuracy:{metric.result().numpy()} Loss:{loss.numpy()}")
            
#     print(f"Epoch:{epoch+1} completed...")
#     print(f"Epoch:{epoch+1}  Accuracy:{metric.result().numpy()}")
        